## Exploring The Telenor Telco Dataset

In [ ]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
import pickle
from datetime import datetime
from sklearn.metrics import f1_score, precision_score, recall_score
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go
import seaborn as sns
import cufflinks as cf
import glob
import sys
import json
from tqdm.notebook import tqdm
cf.go_offline()
init_notebook_mode


### Loading the data

In [ ]:
data_path = '../../Data/Data_DK/radio_kpis_sample.csv'
#data_path = '../../Hackathon2021_Telenor/hackathon_kpis_anonymised.csv'
df = pd.read_csv(data_path, sep=";")
df_orig = df.copy()

In [ ]:
df_orig.head()

### Some info
From below we see that we have 1048575 rows of data, each with 24 columns. 

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.describe()

### Print number of nans for each column

In [ ]:
nan_values = df.isna().sum()
print(nan_values)
nan_columns = nan_values[nan_values > 0].index.tolist()
nan_columns

In [ ]:
df.head()

In [ ]:
# Removing data for a TEST cell 
df = df[df['cell_name'] != 'J5002D11_TEST']

# Converting a OBJECT column to FLOAT
df['thp_nom_tt_kpi'].replace('0,00E+00', 0.0, inplace=True)
df['thp_denom_tt_kpi'].replace('0,00E+00', 0.0, inplace=True)

df['thp_nom_tt_kpi'] = df['thp_nom_tt_kpi'].astype(float)
df['thp_denom_tt_kpi'] = df['thp_denom_tt_kpi'].astype(float)

# Change sector values and carrier
df['sector'] = df['sector'].apply(lambda l: int(l[-1]))
df['cell_name'] = df['cell_name'].apply(lambda l: int(l[-2]))
df = df.rename(columns={'cell_name': 'carrier'})


In [ ]:
#df['timestamp'] = df['period_start_time'].apply(lambda l: str(l[:10]) + ' ' + str(l[11:19]))
df['day_partition_key'] = df['day_partition_key'].astype(str)
df['cell_hour'] = df['cell_hour'].astype(str)
#df['cell_hour'] = df['cell_hour'].apply(lambda l: '0' + l if len(l) == 1 else l)

df['timestamp'] = df['day_partition_key'] + df['cell_hour']
df.head()

### Sorting data after timestamp

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d%H')
df = df.sort_values(by=['timestamp'])
df.tail()

### Checking the resolution 
From below we see that it is hourly, and only once is there a gap of more than an hour (2020-03-29).

In [ ]:
grouped_df = df.groupby(['timestamp'])

for timestamp, group in grouped_df:
    t = timestamp
    break

for timestamp, group in grouped_df:
    next_t = timestamp
    td = next_t - t
    td_mins = int(td.total_seconds() / 60)
    if td_mins != 60:
        print(f'Time between {t} and {timestamp} is not 1 hour, but {td_mins/60} hours.')
    t = next_t


### Checking number of cells that has data present for each timestamp

We see that the number of cells present varies a lot and flunctuates around ~100.

In [ ]:
timestamps = []
num_cells_at_time = []
for timestamp, group in grouped_df:
    timestamps.append(timestamp)
    num_cells_at_time.append(len(group))

num_cell_df = pd.DataFrame({'timestamp': timestamps, 'num_cells': num_cells_at_time})
fig = px.line(num_cell_df, x="timestamp", y='num_cells', title='Number of cells for each timestamp')
fig.show()

plt.boxplot(num_cells_at_time);
plt.title('Boxplot with number of cells for each timestamp')
plt.ylabel('# of cells')
plt.xticks([], [])
plt.show()

## Aggregating cells within same sector

### For Hackaton data
The cell_name is of the form 'XX_ija', where:

- XX in {00,01,02,..,30} denotes the site the cell belongs to;
- i in {1,2,3} denotes the sector the cell belongs to;
- j in {1,2,...} denotes the carrier;
- a in {'Z','X','Y','W','V','R','Q','P'} denotes the technology and frequency of the cell based on the table below.

### For Original data
The cell_name is of the form: '{1char}{4digits}{1char}{2chars}', where:

- 1char is Area
- 4digits is range per Area
- 1char is System (frequency/technology)
- 2chars: 1st digit is carrier layer, starting from 1. 2nd digit is sector, range 0-9

In [ ]:
df.head()

In [ ]:
is_original_data = True
df2 = df.copy()

if not is_original_data:    
    df2['site'] = df2['cell_name'].apply(lambda s: int(s.split('_')[0]))
    df2['sector'] = df2['cell_name'].apply(lambda s: int(s.split('_')[1][0]))
    df2['carrier'] = df2['cell_name'].apply(lambda s: int(s.split('_')[1][1]))
    df2['tech'] = df2['cell_name'].apply(lambda s: s.split('_')[1][2])
df2.head()

### Fill NaN using mean or median of column within same sector

In [ ]:
df2.info()

In [ ]:
agg_df = df2.copy()
for c in nan_columns:
    if is_numeric_dtype(agg_df[c]):
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['timestamp', 'site', 'sector'])[c].transform('median'))
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['timestamp', 'site'])[c].transform('median'))
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['site', 'sector'])[c].transform('median'))
        agg_df[c] = agg_df[c].fillna(agg_df.groupby(['site'])[c].transform('median'))
        agg_df[c] = agg_df[c].fillna(agg_df[c].median())

print(agg_df.isna().sum())
agg_df.head()

### Now aggregate across each sector

In [ ]:
agg_df.columns

In [ ]:
# To aggregate columns differently, pass this to the agg-function
#aggregate_methods = {
 #   'avail_period_duration': 'mean',
  #  'unavail_unplan_nom': 'mean',
   # 'unavail_unplan_denom': 'mean',
    #'unavail_total_nom': 'mean',
    #'unavail_total_denom': 'mean',
    #'bandwidth': 'mean'
#}

agg_df = agg_df.groupby(['timestamp', 'site', 'sector']).agg('mean').reset_index()
# Remove irrelevant columns 
agg_df.drop(columns=['gid', 'carrier'], inplace=True)
agg_df[(agg_df['site'] == 'J1824') & (agg_df['sector'] == 1)].head()

In [ ]:
grouped_agg_df = agg_df.groupby(['timestamp'])
timestamps = []
num_sectors_at_time = []
for timestamp, group in grouped_agg_df:
    timestamps.append(timestamp)
    num_sectors_at_time.append(len(group))

num_sector_df = pd.DataFrame({'timestamp': timestamps, 'num_sectors': num_sectors_at_time})
fig = px.line(num_sector_df, x="timestamp", y='num_sectors', hover_data={"timestamp": "|%B %d. %H:%M, %Y"}, title='Number of sectors for each timestamp')
fig.show()


plt.boxplot(num_sectors_at_time);
plt.title('Boxplot with number of sectors for each timestamp')
plt.ylabel('# of sectors')
plt.xticks([], [])
plt.show()

### Visualizing the number of cells in each sector, for each site

In [ ]:
plt.figure(figsize=(70, 10))
ax = sns.countplot(x="site", hue="sector", data=agg_df)
plt.legend(loc='upper right', fontsize=40)
ax.set_ylabel('# of cells', fontsize=50)
ax.xaxis.label.set_size(50)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.title('Number of cells within each sector, for each site', fontsize=70)
plt.show()

### Visualizing data for one of the sectors

In [ ]:
def visualize_sector(df, site, sector, column):
    sector_df = df[(df['site'] == site) & (df['sector'] == sector)]
    fig = px.line(sector_df, x="timestamp", y=column)
    fig.show()

In [ ]:
visualize_sector(agg_df, 'J0847', 1, 'mcdr_denom')

## Creating dataset 
In **agg_df**, each row corresponds to aggregated data from all cells within a sector, at a specific timestamp. However, not all sectors do have at least one active cell at all timestamps. If a sector contains no active cells for a timestamp, then a row for that sector at that timestamp does not exist. To make the resolution (every hour) and shape (there is a row for all sectors for all timestamps) consistent, we must insert rows. 

In [ ]:
agg_df.columns

In [ ]:
default_values = {
    'avail_period_duration': 60,
    'unavail_unplan_nom': 0,
    'unavail_unplan_denom': 60,
    'unavail_total_nom': 0,
    'unavail_total_denom': 60,
    'bandwidth': 0,
    'mcdr_denom': 0, 
    'mcdr_nom_s': 0,
    'mcdr_nom_d': 0,
    'msdr_denom': 0,
    'msdr_nom_s': 0,
    'msdr_nom_d': 0,
    'thp_denom_tt_kpi': 0,
    'thp_nom_tt_kpi': 0,
    'ho_denom': 0,
    'ho_nom': 0
}

default_values_list = [default_values[c] for c in agg_df.columns[3:]]

In [ ]:
is_present_set = set([tuple(x) for x in agg_df[['timestamp', 'site', 'sector']].values])

In [ ]:
all_timestamps

In [ ]:
datetime_start = agg_df.head(1)['timestamp'].iloc[0]
datetime_end = agg_df.tail(1)['timestamp'].iloc[0]

all_timestamps = pd.date_range(datetime_start, datetime_end, freq="H")
all_sites = agg_df['site'].unique()
all_sectors = agg_df['sector'].unique()

rows_to_add = []
for timestamp in tqdm(all_timestamps):
    for site in all_sites:
        for sector in all_sectors:
            if (timestamp, site, sector) not in is_present_set:
                row_insert = [timestamp, site, sector]
                row_insert.extend(default_values_list)
                rows_to_add.append(row_insert)

In [ ]:
rows_to_add_df = pd.DataFrame(rows_to_add, columns=agg_df.columns)

In [ ]:
agg_df_filled = agg_df.append(rows_to_add_df)
agg_df_filled = agg_df_filled.sort_values(by=['timestamp', 'site', 'sector'])

In [ ]:
print(agg_df_filled.shape)
agg_df_filled.head(6)

#### Check that new dataframe now has constant number of sectors for each timestamp

In [ ]:
grouped_agg_df = agg_df_filled.groupby(['timestamp'])
timestamps = []
num_sectors_at_time = []
for timestamp, group in grouped_agg_df:
    timestamps.append(timestamp)
    num_sectors_at_time.append(len(group))

num_sector_df = pd.DataFrame({'timestamp': timestamps, 'num_sectors': num_sectors_at_time})
fig = px.line(num_sector_df, x="timestamp", y='num_sectors', hover_data={"timestamp": "|%B %d. %H:%M, %Y"}, title='Number of sectors for each timestamp')
fig.show()

#### Visualize column again

In [ ]:
visualize_sector(agg_df_filled, 'J0847', 1, 'mcdr_denom')

### Now we group the data to get a series of graph signals with shape: (# of timestamps, # of sectors, # of features)

<img src="https://i.imgur.com/1izToWi.png" width=750 height=750 />

In [ ]:
grouped_agg_df = agg_df_filled.groupby(['timestamp'])
graph_signals = []

for timestamp, group in tqdm(grouped_agg_df):
    graph_signals.append(group)

graph_signals = np.array(graph_signals)

metadata = {
    'start': str(agg_df_filled.head(1)['timestamp'].iloc[0]),
    'end': str(agg_df_filled.tail(1)['timestamp'].iloc[0]),
    'resolution_minutes': (list(grouped_agg_df)[1][0] - list(grouped_agg_df)[0][0]).seconds / 60,
    'columns': agg_df_filled.columns[1:].tolist(),
}

In [ ]:
print(metadata)
print(graph_signals.shape)

with open('../datasets/telenor/metadata.txt', 'w') as file:
    json.dump(metadata, file, indent=2)
    
np.save('../datasets/telenor/graph_signals.npy', graph_signals)

In [ ]:
sector_df_processed = sector_df.copy().drop(columns=['timestamp', 'site', 'sector'])

###  Save data for each sector as separate numpy files

In [ ]:
def get_sector_data(df, site, sector, test_split=0.3):
    data = df.copy()
    sector_data = data[(data['site'] == site) & (data['sector'] == sector)]
    sector_data = sector_data.sort_values(by=['timestamp'])
    sector_data_processed = sector_data.drop(columns=['timestamp', 'site', 'sector', 'carrier'])
    train_idx_stop = int(sector_data_processed.shape[0] * (1-test_split))
    train = sector_data_processed.iloc[:train_idx_stop, :]
    test = sector_data_processed.iloc[train_idx_stop:, :]
    return train, test

In [ ]:
save_data = False
if save_data:
    train_folder = '../datasets/telenor/train'
    test_folder = '../datasets/telenor/test'
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    labeled_anomalies = {'chan_id': [],
                         'anomaly_sequences': [],
                         'class': [],
                         'num_values': []}

    site_sector_combinations = agg_df[['site', 'sector']].value_counts().index.tolist()
    for site, sector in sorted(site_sector_combinations):
        train, test = get_sector_data(agg_df, site, sector, test_split=0.3)
        np.save(f'{train_folder}/{site}-{sector}.npy', train)
        np.save(f'{test_folder}/{site}-{sector}.npy', test)

        labeled_anomalies['chan_id'].append(f'{site}-{sector}')
        labeled_anomalies['anomaly_sequences'].append([])
        labeled_anomalies['class'].append('')
        labeled_anomalies['num_values'].append(test.shape[0])

    labeled_anomalies_df = pd.DataFrame(labeled_anomalies)
    labeled_anomalies_df.to_csv('../datasets/telenor/labeled_anomalies.csv')
    
    train.head()


In [ ]:
sector_df_processed.head()